In [ ]:
# colab에서 실행되었습니다

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import datetime

import torch
import re
import os
import shutil
import glob
import zipfile
import cv2

import tensorflow as tf
from tensorflow import keras

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device :', device)

device : cpu


# 1. 구글 드라이브 마운트

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 작업 경로 변경

# 현재 작업 디렉토리 확인
print('현재 작업 경로 :', os.getcwd())

# 작업 디렉토리 변경
os.chdir('/content/drive/MyDrive/BoneAge')
print('변경된 작업 경로 :', os.getcwd())

현재 작업 경로 : /content
변경된 작업 경로 : /content/drive/MyDrive/BoneAge


# 2. 데이터 확인 및 전처리

In [ ]:
# Height_Korea_2017.xlsx : 소아청소년성장도표(2017)
# https://www.kdca.go.kr/contents.es?mid=a20303030400

height_2017 = pd.read_excel('Height_Korea_2017.xlsx')

# excel파일 csv파일로 저장
height_2017.to_csv('Height_Korea_2017.csv', index=False)

In [ ]:
height_2017.head()

,성별,만나이(세),만나이(개월),L,M,S,1st,3rd,5th,10th,...,95th,97th,99th,-3SD,-2SD,-1SD,0,+1SD,+2SD,+3SD
0,1,0.0,0,1.0,49.8842,0.0380,45.5,46.3,46.8,47.5,...,53.0,53.4,54.3,44.2,46.1,48.0,49.9,51.8,53.7,55.6
1,1,NaN,1,1.0,54.7244,0.0356,50.2,51.1,51.5,52.2,...,57.9,58.4,59.3,48.9,50.8,52.8,54.7,56.7,58.6,60.6
2,1,NaN,2,1.0,58.4249,0.0342,53.8,54.7,55.1,55.9,...,61.7,62.2,63.1,52.4,54.4,56.4,58.4,60.4,62.4,64.4
3,1,NaN,3,1.0,61.4292,0.0333,56.7,57.6,58.1,58.8,...,64.8,65.3,66.2,55.3,57.3,59.4,61.4,63.5,65.5,67.6
4,1,NaN,4,1.0,63.8860,0.0326,59.0,60.0,60.5,61.2,...,67.3,67.8,68.7,57.6,59.7,61.8,63.9,66.0,68.0,70.1


In [ ]:
# 데이터 전처리

height_2017 = pd.read_csv('Height_Korea_2017.csv')
height_2017 = height_2017.rename(columns={'만나이(세)':'AGE','만나이(개월)':'MONTH','성별':'GENDER'})
male_df = height_2017[height_2017['GENDER'] == 1]
female_df = height_2017[height_2017['GENDER'] == 2]

male_year_3 = male_df[male_df['AGE'] >=3]
female_year_3 = female_df[female_df['AGE'] >=3]

male_year = male_year_3.drop(['GENDER','L','M','S','-3SD','-2SD','-1SD', '0','+1SD','+2SD','+3SD'], axis=1)
male_year = male_year.set_index('AGE')

female_year = female_year_3.drop(['GENDER','L','M','S','-3SD','-2SD','-1SD', '0','+1SD','+2SD','+3SD'], axis=1)
female_year = female_year.set_index('AGE')

In [ ]:
male_year.head()

,MONTH,1st,3rd,5th,10th,15th,25th,50th,75th,85th,90th,95th,97th,99th
AGE,,,,,,,,,,,,,,
3.0,36,88.3,89.7,90.5,91.8,92.6,93.9,96.5,99.2,100.7,101.8,103.4,104.4,106.5
4.0,48,94.0,95.6,96.5,97.9,98.9,100.3,103.1,105.9,107.5,108.5,110.1,111.2,113.2
5.0,60,99.7,101.6,102.5,104.1,105.1,106.7,109.6,112.6,114.2,115.3,116.9,118.0,120.0
6.0,72,105.5,107.4,108.4,110.1,111.2,112.8,115.9,119.1,120.8,122.0,123.8,125.0,127.2
7.0,84,111.0,113.1,114.2,115.9,117.0,118.8,122.1,125.4,127.3,128.6,130.5,131.7,134.1


In [ ]:
# DataFrame 이름 변경

lms_df = height_2017
df_m = male_year
df_fm = female_year

In [ ]:
lms_df.head()

# 1st ~ 99th : 신장(cm) 백분위수

,GENDER,AGE,MONTH,L,M,S,1st,3rd,5th,10th,...,95th,97th,99th,-3SD,-2SD,-1SD,0,+1SD,+2SD,+3SD
0,1,0.0,0,1.0,49.8842,0.0380,45.5,46.3,46.8,47.5,...,53.0,53.4,54.3,44.2,46.1,48.0,49.9,51.8,53.7,55.6
1,1,NaN,1,1.0,54.7244,0.0356,50.2,51.1,51.5,52.2,...,57.9,58.4,59.3,48.9,50.8,52.8,54.7,56.7,58.6,60.6
2,1,NaN,2,1.0,58.4249,0.0342,53.8,54.7,55.1,55.9,...,61.7,62.2,63.1,52.4,54.4,56.4,58.4,60.4,62.4,64.4
3,1,NaN,3,1.0,61.4292,0.0333,56.7,57.6,58.1,58.8,...,64.8,65.3,66.2,55.3,57.3,59.4,61.4,63.5,65.5,67.6
4,1,NaN,4,1.0,63.8860,0.0326,59.0,60.0,60.5,61.2,...,67.3,67.8,68.7,57.6,59.7,61.8,63.9,66.0,68.0,70.1


# 3. 성장 예측 모델 생성

* 2017 소아성장도표 데이터 사용
* 측정된 골연령(BA)과 현재 신장(current_H)를 적용해 100분위 수 측정
* 만 18세의 성장도표에서 측정된 100분위수를 성장 예측에 적용


In [ ]:
# 신장 예측

def Height_prediction (gender, BA, current_H, lms_df):
    month_age = round(BA * 12)
    if gender == 1:
        lms_index = month_age
        L_18, M_18, S_18 = lms_df.iloc[227,3], lms_df.iloc[227,4], lms_df.iloc[227,5]
    elif gender == 0:
        lms_index = month_age
        L_18, M_18, S_18 = lms_df.iloc[455,3], lms_df.iloc[455,4], lms_df.iloc[455,5]

    L,M,S = lms_df.iloc[lms_index,3], lms_df.iloc[lms_index,4],lms_df.iloc[lms_index,5]
    x = current_H

    Z = (((x/M)**L)-1)/(L*S)
    Z = round(Z,4)

    pred_height = M_18 * (1 + (L_18 * S_18 * Z)) ** (1 / L_18)
    pred_height = round(pred_height, 1)
    return pred_height

In [ ]:
# 백분위 찾기

def find_th(df,BA,Height):
    df = df.reset_index()
    find_df = df[(df['AGE'] >= BA) & (df['MONTH'] >= BA*12 )] >= Height
    cdf = df[(df['AGE'] >= BA) & (df['MONTH'] >= BA*12 )]
    find_df = find_df.drop('MONTH',axis=1)
    find_df = find_df.iloc[0]
    find_df = find_df[find_df==True].index
    try:
        result_th = list(find_df)[0]

        if cdf.loc[(cdf.index[0]) , '1st'] - 1.5 > Height :
            result_th = 'Abnormal'

        else:
            pass
    except:
        if cdf.loc[(cdf.index[0]) , '99th'] + 1.5 >= Height :
            result_th = '99th'
        else:
            result_th='Abnormal'

    return result_th

In [ ]:
# 샘플 데이터로 확인

gender = 1
BA = 3.0
current_H = 96.5

df_m = male_year
df_fm = female_year
lms_df = pd.read_csv('Height_Korea_2017.csv')

if gender == 1:
  df = male_year
elif gender == 0:
  df = female_year

def Prediction(gender, BA, current_H, df):
  prediction = Height_prediction(gender, BA, current_H, lms_df)
  result_th = find_th(df, BA, current_H)
  print(f"Bone Age : {BA} & Current Height : {current_H} // {result_th}")
  print(f">>> Prediction Height : {prediction} // {result_th}")

Prediction(gender, BA, current_H, df)


Bone Age : 3.0 & Current Height : 96.5 // 50th
>>> Prediction Height : 174.5 // 50th
